# Continuous-Variable QAOA with Strawberry Fields on Xanadu Backend Simulator "simulon_gaussian"

This notebook is adapted from the original implementation shared at  
[github.com/Lucaman99/Quantum-Computing](https://github.com/Lucaman99/Quantum-Computing/blob/master/xanadu/cvqaoa.ipynb),  
which demonstrates the **Quantum Approximate Optimization Algorithm (QAOA)** in the **continuous-variable (CV)** setting using **Strawberry Fields**, a photonic quantum computing platform developed by Xanadu.

In the original notebook, all simulations are run locally using built-in simulators such as `fock` and `gaussian`.  
In this adapted version, we make minor modifications to the original code in order to run the algorithm on **Xanadu's cloud-based backend**, specifically the `simulon_gaussian` backend. This allows for more realistic simulations of photonic hardware and prepares the circuit for future execution on actual quantum devices.

## Goals of This Notebook

- Review the structure of CV-QAOA and its application to optimization problems.
- Construct cost and mixer Hamiltonians suitable for CV systems.
- Modify the QAOA circuit for compatibility with the `simulon_gaussian` backend.
- Observe and analyze the output photon statistics returned by the backend.

We gratefully acknowledge the work of the original author in developing the first version of this notebook.


In [1]:
import strawberryfields as sf
from strawberryfields.ops import *
# from strawberryfields.utils import scale
from numpy import pi, sqrt
import math
import random
from scipy.optimize import minimize
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.integrate import simps

## CV-QAOA for a Simple Quadratic Cost Function

We simulate QAOA for the cost function  
$$f(x) = (x - a)^2$$  
which translates to the cost Hamiltonian:  
$$\hat{H}_C = (\hat{x} - a)^2$$

The QAOA unitary is:  
$$U(\alpha, \gamma) = e^{-i\alpha \hat{H}_M} \, e^{-i\gamma (\hat{x} - a)^2}$$  
where $\hat{H}_M$ is a mixer Hamiltonian (e.g., kinetic energy or number operator).

Expanding the cost unitary:  
$$e^{-i\gamma(\hat{x} - a)^2} = e^{-i\gamma \hat{x}^2} \, e^{i 2a\gamma \hat{x}} \cdot e^{-i\gamma a^2}$$  
We drop the global phase $e^{-i\gamma a^2}$.

These operators correspond to:
- **Quadratic phase gate**:  
  $$P(s) = \exp\left(i \frac{s}{2\hbar} \hat{x}^2\right), \quad s = -2\gamma\hbar$$
- **Momentum displacement gate**:  
  $$Z(p) = \exp\left(i \frac{p}{\hbar} \hat{x}\right), \quad p = 2\gamma a \hbar$$

We'll implement this using Strawberry Fields, choosing a value like $a = 0.83$.

> **Note**: For a more in-depth explanation of the derivation and theory, please refer to the original notebook:  
> [https://github.com/Lucaman99/Quantum-Computing/blob/master/xanadu/cvqaoa.ipynb](https://github.com/Lucaman99/Quantum-Computing/blob/master/xanadu/cvqaoa.ipynb)


#### Attempt to Run on Xanadu X8 Hardware

As part of this project, I attempted to implement the CV-QAOA algorithm on **Xanadu’s X8 hardware** — the only available photonic quantum chip on their cloud platform.

However, through discussions with Xanadu’s support team, I learned that the **X8 is a passive device** and **cannot implement displacement operations**, which are required for the cost unitary  
$$e^{i p \hat{x} / \hbar}.$$


##### Why It Cannot Be Implemented on X8

The unitary  
$$e^{i p \hat{x} / \hbar}$$  
**cannot be implemented** on the X8 chip because:

- X8 only supports **passive Gaussian operations** (unitaries on annihilation operators: $\hat{a} \mapsto U\hat{a}$).
- Passive operations **preserve the origin** in phase space (they can rotate or squeeze, but **not displace**).
- Displacements are **active transformations** — they change the **first moments** in phase space.
- Active operations like displacement require **external coherent drives** or **electro-optic modulation**, which **X8 does not support**.


##### Could It Be Emulated Indirectly?

A common question is whether clever combinations of passive components (e.g., interferometers, beam splitters, or phase shifters) could fake a displacement.

**The answer is no**:

- A displacement **cannot** be emulated with interferometers, beam splitters, or phase shifters alone.
- Even with **multiple modes**, there's no way to generate a displacement without **injecting an external coherent state or ancilla** — which X8 does not permit.



#### Transition to `simulon_gaussian` Backend and the `Pgate` Issue

After realizing that Xanadu’s **X8 hardware** does not support displacement operations due to its passive nature, I decided to move forward with the **`simulon_gaussian`** backend — a **simulator** that supports a broader set of Gaussian operations.

However, a new limitation emerged:  
Although `simulon_gaussian` supports **displacement and squeezing**, it **does not support the `Pgate`**, which is essential for implementing the quadratic phase  
$$\hat{P}(s) = \exp\left( i \frac{s}{2\hbar} \hat{x}^2 \right).$$

---

#####  Workaround: Emulating `Pgate` Using Conjugation

To overcome this, I used a known decomposition based on **conjugation with rotation gates**. Specifically, we use the identity:

$$
\hat{P}(s) = \hat{R}(-\pi/2) \, \hat{Z}(s) \, \hat{R}(\pi/2)
$$

Here’s what this means:

- **`Pgate(s)`** applies a momentum shear and is equivalent to a **quadratic phase in position**.
- **`Zgate(z)`** performs a **momentum displacement**:  
  $$\hat{Z}(z) = \exp(i z \hat{x})$$
- **`Rgate(θ)`** rotates the phase space:  
  $$\hat{R}(θ) = \exp(i θ \hat{n})$$  
  and acts on quadratures as:
  $$
  \begin{aligned}
  \hat{a} &\rightarrow \hat{a} \cos θ + \hat{a}^\dagger \sin θ \\
  \hat{p} &\rightarrow \hat{p} \cos θ - \hat{x} \sin θ
  \end{aligned}
  $$

While neither `Rgate` nor `Zgate` alone can replicate a `Pgate`, their **conjugation** can.

---

#####  Code Implementation

```python
Rgate(-np.pi/2) | q[0]
Zgate(-2 * hbar * p) | q[0]
Rgate(np.pi/2) | q[0]


## Now let's pick our remote engine and start the algorithm

In [2]:
eng = sf.RemoteEngine("simulon_gaussian")

In [3]:
#Defines the value of h-bar

sf.hbar = 0.5
hbar = 0.5

#Defines the value of the parabolic minimum

parabolic_min = 0.83

In [4]:
#Defines the classical cost function

def function_optimize(x, parabolic_min):
    y = (x - parabolic_min)**2
    return y

In [5]:
#Defines the cost unitary for QAOA

def cost_ham(q, p):

    Zgate(parabolic_min*2*hbar*p) | q[0]
    # Pgate(-2*hbar*p) | q[0]
    Rgate(-np.pi/2) | q[0]
    Zgate(-2 * hbar * p) | q[0]
    Rgate(np.pi/2)  | q[0]

In [6]:
#Defines the kinetic mixer unitary for QAOA

def mixer_ham(q, p):

    Rgate(-1*math.pi/2) | q[0]
    # Pgate(-1*hbar*p) | q[0]
    Rgate(-np.pi/2)     | q[0]
    Zgate(-hbar * p)    | q[0]
    Rgate(np.pi/2)      | q[0]

    Rgate(math.pi/2) | q[0]

    return q 

In [7]:
#Defines the circuit depth, cutoff dimension, and the squeezing parameter

circuit_depth = 3
squeezing = -0.5
cutoff = 5

In [8]:
#Defines the function used to run the circuit and return numerical data

def run_circuit(param):

    # eng = sf.Engine("fock", backend_options={"cutoff_dim":cutoff})    # we used remoteEngine above 'the backend simulator'
    prog = sf.Program(1)

    param1 = [param[0] for i in range(0, circuit_depth)]
    param2 = [param[0] for i in range(0, circuit_depth)]

    with prog.context as q:

        Squeezed(squeezing,0) | q[0]

        for i in range(0, circuit_depth):

            cost_ham(q, param1[i])
            mixer_ham(q, param2[i])

    result = eng.run(prog, shots=100) # run_options={"eval": False})
    state = result.state
    a = state.wigner(0, [i/10 for i in range(-30, 30)], [i/10 for i in range(-30, 30)])
    return a

In [9]:
#Defines the sampling circuit simulation

def old_circuit(param):

    # eng = sf.Engine("fock", backend_options={"cutoff_dim":cutoff})
    prog = sf.Program(1)

    param1 = [param[0] for i in range(0, circuit_depth)]
    param2 = [param[0] for i in range(0, circuit_depth)]

    with prog.context as q:

        Squeezed(squeezing,0) | q[0]

        for i in range(0,circuit_depth):

            cost_ham(q, param1[i])
            mixer_ham(q, param2[i])

        MeasureX | q[0]

    result = eng.run(prog, shots=1)
    return result.samples[0][0]

In [10]:
#Defines the number of iterations that the circuit is sampled

shots = 15

In [11]:
#Defines the objective function that is minimized

def objective(param):

    costly = 0

    for i in range(0, shots):

        av = old_circuit(param)
        result1 = av
        calculation = function_optimize(result1, parabolic_min)
        costly = costly + calculation

    costly = costly/shots


    print("Paramter: "+str(param))
    print("Value of Cost Function: "+str(costly))

    return costly

In [12]:
#Defines the optimizer that minimizes the objective function

out = minimize(objective, x0=[random.randint(-150,150)/100], method="Nelder-Mead", options={'maxiter':20})
print(out)

2025-07-07 23:42:57,522 - INFO - Compiling program for device simulon_gaussian using compiler gaussian.
2025-07-07 23:43:00,723 - INFO - The remote job 131796ea-f5d7-4fb7-a6f7-d419fc099e1d has been completed.
2025-07-07 23:43:01,068 - INFO - Compiling program for device simulon_gaussian using compiler gaussian.
2025-07-07 23:43:06,470 - INFO - The remote job a9cc250c-cea3-4464-99e4-aa313b164a92 has been completed.
2025-07-07 23:43:06,778 - INFO - Compiling program for device simulon_gaussian using compiler gaussian.
2025-07-07 23:43:10,001 - INFO - The remote job b598c3fa-0aa9-4a51-84f0-82cef67476fa has been completed.
2025-07-07 23:43:10,333 - INFO - Compiling program for device simulon_gaussian using compiler gaussian.
2025-07-07 23:43:14,660 - INFO - The remote job 41220ef7-6e42-41d1-9a2b-118ce783022d has been completed.
2025-07-07 23:43:15,037 - INFO - Compiling program for device simulon_gaussian using compiler gaussian.
2025-07-07 23:43:18,629 - INFO - The remote job 0dc1ef63-953

Paramter: [-0.27]
Value of Cost Function: 15.285667667709474


2025-07-07 23:44:05,703 - INFO - The remote job df1b6070-1bc2-44f1-8e62-6f66e835792c has been completed.
2025-07-07 23:44:06,034 - INFO - Compiling program for device simulon_gaussian using compiler gaussian.
2025-07-07 23:44:10,403 - INFO - The remote job f5dd3909-7dd1-49b7-9d7c-3202b59c5131 has been completed.
2025-07-07 23:44:10,736 - INFO - Compiling program for device simulon_gaussian using compiler gaussian.
2025-07-07 23:44:13,953 - INFO - The remote job 40e5c886-c993-4e05-af23-af399aeb5d25 has been completed.
2025-07-07 23:44:14,267 - INFO - Compiling program for device simulon_gaussian using compiler gaussian.
2025-07-07 23:44:18,749 - INFO - The remote job 7ee6c488-84a3-40d7-b639-65578de63b49 has been completed.
2025-07-07 23:44:19,117 - INFO - Compiling program for device simulon_gaussian using compiler gaussian.
2025-07-07 23:44:23,493 - INFO - The remote job 655b5299-c3e1-4751-8138-4885a2884310 has been completed.
2025-07-07 23:44:23,869 - INFO - Compiling program for devi

Paramter: [-0.2835]
Value of Cost Function: 15.925598277235657


2025-07-07 23:45:11,972 - INFO - The remote job a2c2b706-f8d0-450c-8707-1602007feecc has been completed.
2025-07-07 23:45:12,274 - INFO - Compiling program for device simulon_gaussian using compiler gaussian.
2025-07-07 23:45:16,675 - INFO - The remote job 05deeeae-e03a-4ad5-af24-73a2997f1328 has been completed.
2025-07-07 23:45:16,995 - INFO - Compiling program for device simulon_gaussian using compiler gaussian.
2025-07-07 23:45:21,214 - INFO - The remote job 0043dd74-f06d-4628-8dc9-fbb63627a117 has been completed.
2025-07-07 23:45:21,528 - INFO - Compiling program for device simulon_gaussian using compiler gaussian.
2025-07-07 23:45:25,753 - INFO - The remote job d97988c1-b269-4a2b-8a7a-459959c620b2 has been completed.
2025-07-07 23:45:26,059 - INFO - Compiling program for device simulon_gaussian using compiler gaussian.
2025-07-07 23:45:28,777 - INFO - The remote job b3e2a73f-12fa-4e90-9dfd-15a25861e77b has been completed.
2025-07-07 23:45:29,051 - INFO - Compiling program for devi

...

Paramter: [0.38169141]
Value of Cost Function: 2.1113088313210147e-07


2025-07-08 00:20:00,480 - INFO - The remote job 95fec134-fa90-48c0-b602-062471ddb1ac has been completed.
2025-07-08 00:20:00,769 - INFO - Compiling program for device simulon_gaussian using compiler gaussian.
2025-07-08 00:20:05,605 - INFO - The remote job 354cca7b-055b-4b15-ba47-71b301b02f0d has been completed.
2025-07-08 00:20:05,933 - INFO - Compiling program for device simulon_gaussian using compiler gaussian.
2025-07-08 00:20:10,374 - INFO - The remote job 729ee0e6-b5d3-40b4-82eb-083a416c2737 has been completed.
2025-07-08 00:20:10,697 - INFO - Compiling program for device simulon_gaussian using compiler gaussian.
2025-07-08 00:20:15,310 - INFO - The remote job 107ec421-af78-464a-b322-dd2db45ba276 has been completed.
2025-07-08 00:20:15,640 - INFO - Compiling program for device simulon_gaussian using compiler gaussian.
2025-07-08 00:20:20,198 - INFO - The remote job dfc6620c-1516-402d-bb9d-124402729e9b has been completed.
2025-07-08 00:20:20,511 - INFO - Compiling program for devi

Paramter: [0.38148047]
Value of Cost Function: 6.498539487002305e-07


2025-07-08 00:21:09,989 - INFO - The remote job 2f4f2004-f50b-48f6-b48e-6865e625f1f9 has been completed.
2025-07-08 00:21:10,315 - INFO - Compiling program for device simulon_gaussian using compiler gaussian.
2025-07-08 00:21:14,397 - INFO - The remote job efc912e0-8976-4afe-b8dc-d41e28c1c18f has been completed.
2025-07-08 00:21:14,691 - INFO - Compiling program for device simulon_gaussian using compiler gaussian.
2025-07-08 00:21:17,545 - INFO - The remote job 72f25a5c-d4ae-444c-86fb-6662bf9f6077 has been completed.
2025-07-08 00:21:17,857 - INFO - Compiling program for device simulon_gaussian using compiler gaussian.
2025-07-08 00:21:21,991 - INFO - The remote job 1cea1462-3eee-4321-8437-8b0c914ebdaa has been completed.
2025-07-08 00:21:22,284 - INFO - Compiling program for device simulon_gaussian using compiler gaussian.
2025-07-08 00:21:25,114 - INFO - The remote job 5ca614e9-8b82-475b-b6d6-903ead6f2b67 has been completed.
2025-07-08 00:21:25,484 - INFO - Compiling program for devi

Paramter: [0.38163867]
Value of Cost Function: 2.0472904406691124e-08
       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 2.0472904406691124e-08
             x: [ 3.816e-01]
           nit: 18
          nfev: 36
 final_simplex: (array([[ 3.816e-01],
                       [ 3.816e-01]]), array([ 2.047e-08,  3.004e-08]))


**Note:** Unlike the original notebook, I was unable to generate Wigner function visualizations because the Xanadu backend simulator's `result.state` does not currently support this feature. This contrasts with the local simulator, which provides full access to the quantum state and enables Wigner function computations. Due to these limitations, visualizing the Wigner function directly from the backend simulator results is not possible at this time.


In [13]:
print(out['x'])

[0.38163867]
